In [2]:
import cv2 as cv
import numpy as np


In [6]:
# Start by getting the params from my_cam_matrix.npz
npz_calib = np.load('pics/my_cam_matrix.npz')

print(npz_calib)

# Get the camera matrix and distortion coefficients
camera_matrix = npz_calib['mtx']
dist_coeffs = npz_calib['dist']

# Get all jpg files in pics/
import os
pics = [f for f in os.listdir('pics/') if f.endswith('.jpg') and f.startswith('2024')]

# Loop through all the pics
for pic in pics:
    # Read the image
    img = cv.imread('pics/' + pic)
    # Get the shape of the image
    h, w = img.shape[:2]
    # Get the new camera matrix
    # new_camera_matrix, roi = cv.getOptimalNewCameraMatrix(camera_matrix, dist_coeffs, (w, h), 1, (w, h))
    # Undistort the image
    undistorted_img = cv.undistort(img, camera_matrix, dist_coeffs)#, None, new_camera_matrix)
    # Save the undistorted image
    # cv.imwrite('pics/undistorted_' + pic, undistorted_img)
    # Show the undistorted image
    cv.imshow('Undistorted Image', undistorted_img)
    if cv.waitKey(0) & 0xFF == ord('q'):
        break
    
cv.destroyAllWindows()

NpzFile 'pics/my_cam_matrix.npz' with keys: mtx, dist
